In [1]:
// Using Library
// println(java.nio.file.Paths.get("").toAbsolutePath().toString())
// %use @file[[DIR_PATH]/[LIB_NAME].json]

In [2]:
@file:Repository("*mavenLocal")
@file:DependsOn("com.github.holgerbrandl:krangl:0.18.7")

// %use krangl

%use coroutines

%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/gson.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/org_json.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/retrofit.json]
%use @file[D:/Programming/Kotlin/learn-kotlin/dependencies/retrofit_gson_converter.json]

In [3]:
import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.collect
import kotlinx.coroutines.GlobalScope
import kotlinx.coroutines.launch
import okhttp3.ResponseBody
import kotlin.reflect.KClass

In [4]:
data class SearchUser(
    @SerializedName("incomplete_results")
    val incompleteResults: Boolean? = null,
    @SerializedName("items")
    val items: List<User?>? = null,
    @SerializedName("total_count")
    val totalCount: Int? = null
)

data class User(
    @SerializedName("avatar_url")
    val avatarUrl: String? = null,
    @SerializedName("bio")
    val bio: String? = null,
    @SerializedName("blog")
    val blog: String? = null,
    @SerializedName("company")
    val company: Any? = null,
    @SerializedName("created_at")
    val createdAt: String? = null,
    @SerializedName("email")
    val email: Any? = null,
    @SerializedName("events_url")
    val eventsUrl: String? = null,
    @SerializedName("followers")
    val followers: Int? = null,
    @SerializedName("followers_url")
    val followersUrl: String? = null,
    @SerializedName("following")
    val following: Int? = null,
    @SerializedName("following_url")
    val followingUrl: String? = null,
    @SerializedName("gists_url")
    val gistsUrl: String? = null,
    @SerializedName("gravatar_id")
    val gravatarId: String? = null,
    @SerializedName("hireable")
    val hireable: Boolean? = null,
    @SerializedName("html_url")
    val htmlUrl: String? = null,
    @SerializedName("id")
    val id: Int? = null,
    @SerializedName("location")
    val location: String? = null,
    @SerializedName("login")
    val login: String? = null,
    @SerializedName("name")
    val name: String? = null,
    @SerializedName("node_id")
    val nodeId: String? = null,
    @SerializedName("organizations_url")
    val organizationsUrl: String? = null,
    @SerializedName("public_gists")
    val publicGists: Int? = null,
    @SerializedName("public_repos")
    val publicRepos: Int? = null,
    @SerializedName("received_events_url")
    val receivedEventsUrl: String? = null,
    @SerializedName("repos_url")
    val reposUrl: String? = null,
    @SerializedName("site_admin")
    val siteAdmin: Boolean? = null,
    @SerializedName("starred_url")
    val starredUrl: String? = null,
    @SerializedName("subscriptions_url")
    val subscriptionsUrl: String? = null,
    @SerializedName("twitter_username")
    val twitterUsername: Any? = null,
    @SerializedName("type")
    val type: String? = null,
    @SerializedName("updated_at")
    val updatedAt: String? = null,
    @SerializedName("url")
    val url: String? = null
)

In [5]:
interface GithubService {
    @JvmSuppressWildcards
    @GET("search/users")
    suspend fun searchUsers(
        @QueryMap params: Map<String, Any>
    ): Response<SearchUser>

    @GET("users/{id}")
    suspend fun getUser(
        @Path("id") username: String
    ): Response<User>
}

In [6]:
val retrofit = Retrofit
        .Builder().apply {
            baseUrl("https://api.github.com/")
            addConverterFactory(GsonConverterFactory.create(GsonBuilder().create()))
        }.build()
        
val service = retrofit.create(GithubService::class.java)

In [7]:
val handler = CoroutineExceptionHandler { _, exception ->
    println("CoroutineExceptionHandler got $exception with suppressed ${exception.suppressed.contentToString()}")
}

In [8]:
suspend fun searchUsers(params: Map<String, Any>): SearchUser? {
    return try {
        val response = service.searchUsers(params)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

suspend fun getUser(username: String): User? {
    return try {
        val response = service.getUser(username)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

In [9]:
// runBlocking {
//     searchUsers(mapOf("q" to "indra", "page" to 1, "per_page" to 10))
// }

In [10]:
var page = 1
var counter = 10
val listOfUser = mutableListOf<User>()

suspend fun collectUser(page: Int): Boolean {
    val response = searchUsers(mapOf("q" to "indra android", "page" to page, "per_page" to 10))
    println("response : " + response?.items?.size)
    response?.items?.forEach {
        it?.login?.let { username ->
            delay(500)
            getUser(username)
        }?.let { listOfUser.add(it) }
    }
    counter--
    return counter > 0
}

runBlocking {
    while(collectUser(page)) {
        println("page : " + page.toString() + ", counter : " + counter.toString())
        page++
        collectUser(page)
    }
}

response : 10
page : 1, counter : 9
response : 5
response : 5
page : 2, counter : 7
response : 0
response : 0
page : 3, counter : 5
response : 0
response : 0
page : 4, counter : 3
response : 0
response : 0
page : 5, counter : 1
response : 0
response : null


In [18]:
listOfUser.asDataFrame()
    .addColumn("No") { it.rowNumber }
    .rename("name" to "Name")
    .select("No", "Name", "followers", "bio", "login", "location")
    .showRows()

No,Name,followers,bio,login,location
1,Indra Mahkota,23,Android Engineer,indramahkota,"Pontianak, Indonesia"
2,Mochamad Indra Yudha Lakaselindra,8,Android and Game Programmer,IndraYudha22,null
3,Galih Indra Firmansyah,21,Android Development Enthusiasts || Google Certi...,galihif,"Serang, Banten"
4,Indra Deva Aji Zakaria,12,"Informatics '19 Telkom University, Android Flu...",indrad373,Bandung
5,Indranil Chatterjee,5,Android Developer and ML enthusiast.,indracs7,India
6,Indrajit Sarkar,2,Just Started learning Android & Web ;-;,indraskr9,null
7,Indra David,6,Just interest with programming like web and and...,ndraa124,Jakarta
8,Indravardhan Reddy M,0,Android || Machine learning || Web Development,indravardhanreddy,null
9,Indra Pradana A,3,"interest : android dev, web dev",iPradana,Surabaya
10,Indrajit,0,"Having worked for 12+ years in the industry, I ...",indrajitkumar,Bangalore
